## Bag of Trikcs
- Paper :  Bag of Tricks for Image Classification with Convolutional Neural Networks
- 학습에 관여하는 여러가지 요소의 변경이 모델의 정확도에 미치는 영향에 대해 조사
- 여러 방법들을 적용하여 4%이상의 성능 상승을 봄

- 사용된 

## 1. Baseline ResNet 학습 전처리
- 데이터 Training
    - 1. 랜덤 샘플링을 통해 [0, 255] 사이의 이미지를 추출하고 이를 32bit floating point로 변환한다.
    - 2. 영역은 [8%, 100%], 비율은 [3/4, 4/3]로 aspect ratio를 유지하는 사각형 영역을 랜덤하게 crop 한다. 이후, crop한 이미지를 다시 224 x 224 정사각형으로 사이즈를 재조정한다.
    - 3. 50% 확률로 이미지를 수평으로 뒤집기
    - 4. [0.6, 1.4] 사이에서 Scale hue, saturation, brightness 값을 뽑음
    - 5. N(0, 0.1) 분포에서 값을 하나 샘플링하여 PCA noise를 추가
    - 6. RGB 채널에 대해 123.68, 116.799, 103.939를 빼준 다음, 이를 각각 58.393, 57.12, 57.375로 나누어 정규화 작업 진행

- 데이터 Validation
    - aspect ratio를 유지하며, 짧은 축이 256이 되게끔 사이즈를 조절하고, 224 x 224 정사각형 사이즈로 center crop을 수행
    - 2. RGB 채널에 대해 정규화 작업을 진행
    - 3. Validation 단계에서는 그 어떤 random 확장도 수행 X

- 기타 옵션 
    - 모든 Conv layer 및 FC layer는 Xavier 초기화를 수행
    - Optimizer로는 NAG (Nesterov Accelerated Gradient)

<hr>



## 1. Efficient Training
- 딥러닝 관련 하드웨어 성능이 빠르게 성장하면서, **큰 batch size를 사용**하거나, **낮은 numerical precision을 사용**등의 효율적인 학습 방법들이 연구되옴.
- 논문에서는 Large-batch training, Low-precision training 으로 나눠서 방법론을 제시
- Large-batch training에서 동일한 epoch하 **더 큰 배치를 사용하는 경우, 더 작은 배치를 사용할 때보다 validation 정확도가 떨어지는 문제가 발생**

### 방법 1) Linear scaling learning rate
- Large batch size는 기댓값에 별 영향을 못 끼치지만, 분산(variance)을 줄여준다.
- 이는 **분산이 줄어들어 기울기의 noise를 줄여주며, 더 높은 learning rate를 사용할 수 있는 여지를 줌**
- **batch size를 키우면 learning rate도 키우는 것이 좋다**, batch-size = 256 / lr = 0.1
    
### 방법 2) Learning rate warmup
- 학습 초반시 모든 파라미터가 랜덤 값으로 초기화되기에 최종 값과는 거리가 멀다.
- 초반 너무 큰 lr을 사용하면 수치적으로 불안정한 상태를 야기하기에 warm-up 기법을 사용
- **시작 시에는 작은 lr를 사용하고, 훈련 과정이 안정화되었을 때 learning rate 초기값을 사용**
    
![bag_of_tricks_lrscheduler](img/bag_of_tricks_lrscheduler.png)

### 방법 3) Zero γ(감마) in BatchNorm
- **BN에서 x 와 곱해지는 값인 γ(감마)는 β(베타)와 마찬가지로 학습할 수 있는 learnable parameter로 학습 전 initialization 필요**
- mini-batch에 대한 x를 정규화하여 x^로 만들고, Scale transformation을 적용하여 γx^ + β로 변환 = (BN 생성과정) (일반적으로 γ= 1와 β= 0으로 초기화)
- **ResNet 구조와 같이 residual connection이 존재하는 경우 γ= 0 으로 초기화 해주는 것이 학습 초기단계에 안정성을 높임**

### 방법 4) No bias decay
- 모든 BN layer에 대해 γ를 0으로 설정하여 residual block들이 x영역의 값 없이 skip connection 결과만 흘려보내게되고, 따라서 초기에 망이 짧아져 학습이 빨라진다.    
- weight decay는 L2 regularization과 같으며, weight와 bias를 포함한 학습 가능한 모든 파라미터에 적용되는게 일반적
- 논문에 의하면 **weight(conv, fc)에만 decay를 주는 것이 overfitting을 방지하는데 효과적이며, bias, γ, β를 포함한 다른 파라미터에는 적용하지 않는 것을 추천**
    
### 방법 5) Low-precision training

![bag_of_tricks_fp16_fp32](img/bag_of_tricks_fp16_fp32.png)

- Neural Network는 32-bit floating point(FP32) precision을 이용하여 학습이 진행
- 최신 하드웨어에서는 lower precision 계산이 지원되면서 속도에서 큰 이점을 얻음
- FP32에서 FP16으로 precision을 줄이게 되면 수를 표현할 수 있는 범위가 줄어들게 되는 단점이 존재
    - 이를 해결하기 위해 Mixed Precision Training 방식 적용하여 학습

![bag_of_tricks_efficient_training_result](img/bag_of_tricks_efficient_training_result.png)
- 그림 출처 : https://hoya012.github.io/blog/Bag-of-Tricks-for-Image-Classification-with-Convolutional-Neural-Networks-Review/

- 정확도는 0.3% 상승, 학습 속도는 약 3배 빨라지는 장점

<hr>

## 2. Model Tweaks (ResNet 변형 모델 실험)
- ResNet architecture에 약간의 모듈들을 수정하여 성능을 올리는 방법을 제안
- 특정 conv layer의 stride를 바꾸는 등 다소 작은 개선

- ResNet Base line (살짝 변형)
![bag_of_tricks_resnet_base](img/bag_of_tricks_resnet_base.png)



![bag_of_tricks_resnet_b_c_d](img/bag_of_tricks_resnet_b_c_d.png)

- ResNet-B 
    - ResNet의 downsampling block을 변경한 것
    - ResNet-50과 비교하여, downsampling blocks의 Path A에서 더 많은 정보를 얻었으며 validation 정확도를 약 0.5% 가량 개선
    
- ResNet-C
    - Inception-V2에서 제안된 방법을 적용
    - ResNet-C는 7 x 7 conv를 3개의 3 x 3 conv로 바꾸어 이전보다 0.2%가량 상승
    
- ResNet-D : Resnet-B에서 영감을 얻음
    - Downsampling block의 Path B에서 더 많은 정보를 얻어 validation 정확도를 약 0.3%가량 추가로 개선
    
![bag_of_tricks_resnet_b_c_d_result](img/bag_of_tricks_resnet_b_c_d_result.png)

- ResNet-D에서 가장 정확도가 좋았으며, 약간의 FLOPS가 증가하였지만, Top-1의 Accuracy가 거의 1% 증가

<hr>

## 3. Training Refinement
- 정확도를 올리기 위한 4가지 학습 방법 제안

### 방법 1) Cosine Learning Rate Decay
- Paper : SGDR-stochastic gradient de- scent with restarts
- Cosine annealing 기법은 learning rate의 초기값을 0으로 두고 여기에 cos 함수를 적용하여 그다음 배치의 learning rate를 구하는 방법
- 기타 lr_scheduler
    - Exponentially decaying : 30 epochs마다 0.1씩 감소
    - Step decay : 2 epochs마다 0.94씩 감소

### 방법 2) Label Smoothing
- Paper : inception-v2 : Rethinking the inception architecture for computer vision
- 0 대신 작은 값을 갖는 label을 사용하여 라벨을 스무딩함

### 방법 3) Knowledge Distillation
- Paper : Distilling the knowledge in a neural network
- 성능이 좋은 teacher model을 이용하여 student model이 적은 연산 복잡도를 가지면서 teacher model의 정확도를 따라가도록 학습을 시키는 방법

### 방법 4) Mixup Training
- Paper : mixup: Beyond empirical risk minimization
- Data에 dependent한 augmentation 기법이며 두 데이터의 이미지와 label을 각각 weighted linear interpolation 하여 새로운 sample을 생성하는 기법

![bag_of_tricks_mixup_ex](img/bag_of_tricks_mixup_ex.png)


### Training Refinement 실험 결과
- 0.4% ~ 0.7% 까지 정확도 향상

![bag_of_tricks_training_refinement](img/bag_of_tricks_training_refinement.png)



## 참고문헌
- https://hoya012.github.io/blog/Bag-of-Tricks-for-Image-Classification-with-Convolutional-Neural-Networks-Review/
    
- https://phil-baek.tistory.com/entry/CNN-%EA%BF%80%ED%8C%81-Bag-of-Tricks-for-Image-Classification-with-Convolutional-Neural-Networks-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0